# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
!pip install citipy

In [84]:
# Dependencies
import json
import requests
import pandas as pd
from api_keys import weather_api_key, g_key
from citipy import citipy
import numpy as np
from scipy.stats import linregress
from scipy import stats
from pprint import pprint
import os
import datetime as dt

#file; for the end
data = "../Output/cities.csv"



## Generate Cities List

In [85]:
# Ranges of coords
range_lat = (-90, 90)
range_lng = (-180, 180)

# List for holding coords and cities
coordinates = []
cities = []

#gather random coords
np.random.seed(1000)

lats = np.random.uniform(range_lat[0], range_lat[1], size=2000)
longs = lngs = np.random.uniform(range_lng[0], range_lng[1], size=2000)

coordinates = zip(lats, longs)

#Nearest city
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

# If the city is unique, then add it to cities list
    if city not in cities:
        cities.append(city)
        


In [86]:
# Print the city count to confirm sufficient count
len(cities)

759

In [90]:
# Convert city list to dataframe and rename column to "city"
weather_dict = {
    "City": cities,
}
weather_data = pd.DataFrame(weather_dict)

# Add columns to cities dataframe
weather_data["Cloudiness"] = np.nan
weather_data["Country"] = np.nan
weather_data["Date"] = np.nan
weather_data["Humidity"] = np.nan
weather_data["Lat"] = np.nan
weather_data["Lng"] = np.nan
weather_data["Max Temp"] = np.nan
weather_data["Wind Speed"] = np.nan
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,dali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bluff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,norman wells,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tabiauea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,leningradskiy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [94]:
# Save config information #reference
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

city = "London"

# Build query URL | F-strings can be used too.
# query_url = f"{url}""
query_url = url + "appid=" + weather_api_key + "&units=" + units + "&q=" +city


# Get weather data
weather_response = requests.get(query_url)
#convert to a json
weather_json = weather_response.json()

# Get the temperature from the response
print(f"The weather API responded with: {weather_json}.")

The weather API responded with: {'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 42.75, 'feels_like': 42.75, 'temp_min': 38.17, 'temp_max': 46.04, 'pressure': 1013, 'humidity': 93}, 'visibility': 10000, 'wind': {'speed': 1.01, 'deg': 246, 'gust': 4}, 'clouds': {'all': 28}, 'dt': 1622003008, 'sys': {'type': 2, 'id': 2019646, 'country': 'GB', 'sunrise': 1622001276, 'sunset': 1622059251}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}.


In [103]:
weather_json

{'coord': {'lon': 100.3354, 'lat': 5.4112},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02n'}],
 'base': 'stations',
 'main': {'temp': 300.57,
  'feels_like': 304.72,
  'temp_min': 299.81,
  'temp_max': 301.42,
  'pressure': 1008,
  'humidity': 87},
 'visibility': 8000,
 'wind': {'speed': 2.68, 'deg': 90, 'gust': 0},
 'clouds': {'all': 20},
 'dt': 1622569055,
 'sys': {'type': 2,
  'id': 2037578,
  'country': 'MY',
  'sunrise': 1622588660,
  'sunset': 1622633355},
 'timezone': 28800,
 'id': 1735106,
 'name': 'George Town',
 'cod': 200}

In [111]:
weather_json['coord']['lon']
weather_json['coord']['lat']
weather_json['clouds']['all']
weather_json['sys']['country']
weather_json['main']['humidity']
weather_json['main']['temp_max']
weather_json['wind']['speed']
# dt.date(weather_json['dt'])

100.3354

5.4112

20

'MY'

87

301.42

2.68

In [110]:
dt.datetime.fromtimestamp(int(weather_json['dt'])).strftime('%Y-%m-%d %H:%M:%S')

'2021-06-01 12:37:35'

In [114]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"

cities_found = []
cloudiness = []
countries = []
dates = []
humidities = []
lats = []
lngs = []
max_temps = []
wind_speeds = []

# Use iterrows to loop through dataframe using the city name
for city in cities[0:20]:
    # Build query URL | F-strings can be used too.
    # query_url = f"{url}""
    query_url = url + "appid=" + weather_api_key + "&q=" + city.replace(" ","+")

    # Get weather data
    weather_response = requests.get(query_url)
    #convert to a json
    weather_json = weather_response.json()
    try:
        dates.append(dt.datetime.fromtimestamp(int(weather_json['dt'])).strftime('%Y-%m-%d %H:%M:%S'))
        cloudiness.append(weather_json['clouds']['all'])
        countries.append(weather_json['sys']['country'])
        humidities.append(weather_json['main']['humidity'])
        lats.append(weather_json['coord']['lat'])
        lngs.append(weather_json['coord']['lon'])
        max_temps.append(weather_json['main']['temp_max'])
        wind_speeds.append(weather_json['wind']['speed'])
        cities_found.append(city)
    except:
        print("city not found")

    # Get the temperature from the response
    print(f"The weather API responded with: {weather_json}." + "\n")
    
    


The weather API responded with: {'coord': {'lon': 100.1833, 'lat': 25.7}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 290.02, 'feels_like': 290.09, 'temp_min': 290.02, 'temp_max': 290.02, 'pressure': 1008, 'humidity': 89, 'sea_level': 1008, 'grnd_level': 803}, 'visibility': 10000, 'wind': {'speed': 2.53, 'deg': 240, 'gust': 3.1}, 'clouds': {'all': 16}, 'dt': 1622571011, 'sys': {'country': 'CN', 'sunrise': 1622586485, 'sunset': 1622635603}, 'timezone': 28800, 'id': 1814093, 'name': 'Dali', 'cod': 200}.

The weather API responded with: {'coord': {'lon': 168.3333, 'lat': -46.6}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 283.04, 'feels_like': 279.66, 'temp_min': 283.04, 'temp_max': 283.04, 'pressure': 1010, 'humidity': 87, 'sea_level': 1010, 'grnd_level': 1007}, 'visibility': 10000, 'wind': {'speed': 8.07, 'deg': 268, 'gust

The weather API responded with: {'coord': {'lon': 25.9704, 'lat': 70.9821}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 280.01, 'feels_like': 275.53, 'temp_min': 280.01, 'temp_max': 280.01, 'pressure': 1025, 'humidity': 87}, 'visibility': 10000, 'wind': {'speed': 8.75, 'deg': 260}, 'clouds': {'all': 90}, 'dt': 1622571337, 'sys': {'type': 1, 'id': 1638, 'country': 'NO', 'sunrise': 0, 'sunset': 0}, 'timezone': 7200, 'id': 779554, 'name': 'Honningsvåg', 'cod': 200}.

The weather API responded with: {'coord': {'lon': 88.3972, 'lat': 69.4865}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 271.36, 'feels_like': 271.36, 'temp_min': 271.36, 'temp_max': 271.36, 'pressure': 1008, 'humidity': 97, 'sea_level': 1008, 'grnd_level': 999}, 'visibility': 10000, 'wind': {'speed': 0.83, 'deg': 224, 'gust': 1.07}, 'clouds': {'all': 90}, '

In [116]:
(countries)

17

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [46]:
#  Get the indices of cities that have humidity over 100%.


In [47]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression